Import Libraries

In [49]:
from pyvis.network import Network
import networkx as nx
import random

Create Helper Functions

In [50]:
# recursive function that will traverse the nodes
def createPaper(network, authors, probStop):
    '''
    Will take network, list of authors, and probStop as input
    '''
    currAuthorID = authors[-1]
    newNeighbors = set(network.neighbors(currAuthorID)).difference(set(authors))

    # base condition: stop at node if probStop hit or there are no new neighbors to traverse
    if random.random() < probStop or len(newNeighbors) == 0:
        return
    
    # create list reprsenting probabilities for the neighboring nodes of the current coauthor
    probs = []
    for neighbor in newNeighbors:
        nData = network.get_edge_data(currAuthorID, neighbor)
        probs.extend([neighbor] * nData["weight"])

    # Select coauthor from neighbors probabilities list
    try:
        coauthorID = random.choice(probs)
    except IndexError:
        print(f'Probs: {probs} and neighbors: {newNeighbors} and weight: {nData["weight"]}')
        print(network.edges.data())

    # update all edges of coauthors to this new author
    for author in authors:
        # if there is not an edge, create one
        if not network.has_edge(author, coauthorID) and author != coauthorID:
            network.add_edge(author, coauthorID, weight=0, width=1)
        newWeight = network.get_edge_data(author, coauthorID)["weight"] + 1
        #network.update(edges=[ (author, coauthorID, {"weight": newWeight, "width": newWeight//2}) ])
        network.update(edges=[ (author, coauthorID, {"weight": newWeight}) ])

    # call function recursively with coauthor
    authors.append(coauthorID)
    createPaper(network, authors, probStop)

def create():
    return

Define initial parameters

In [51]:
# define time steps
timeSteps = 50

# Probabilities
# probability that you generate new author
pw = 0.5

# define fields
fieldColors = {"CS": "blue", 
                "Math": "green", 
                "Physics": "red"}
fields = list(fieldColors.keys())

# define initial scholars, will be in form (id, scholarField, color)
scholarField = random.choice(fields)
nodeID = 0


Create Model

In [52]:
network = nx.Graph()
scholarField = random.choice(fields)
network.add_node(nodeID, label=scholarField, color=fieldColors[scholarField])
# go through time steps, add new scholar and paper at each step
for i in range(1, timeSteps):

    # Choose first author, either new scholar or random choice
    currNodes = list(network.nodes())
    authors = [random.choice(currNodes)]

    # with probability, add new author to network
    if random.random() < pw:
        # generate author and field
        scholarField = random.choice(fields)
        nodeID += 1
        author = nodeID
        network.add_node(author, label=scholarField, color=fieldColors[scholarField])

        # generate random coauthor from currNodes, which doesn't have the new node added in
        coauthorID = random.choice(currNodes)
        network.add_edge(author, coauthorID, weight=1, width=1)

        # update authors list
        authors = [author, coauthorID]

    # Add new paper, calling function
    probStop = 0.3
    createPaper(network, authors, probStop)

    # add authors as members to this topic

    # add emergence of new topic if field multidiscipiplinary (i.e. the union of the fields)

    # do you make it the author of the paper?

Probs: [] and neighbors: {1, 2, 6} and weight: 0
[(0, 1, {'weight': 19, 'width': 1}), (0, 2, {'weight': 0, 'width': 1}), (0, 3, {'weight': 20, 'width': 1}), (0, 4, {'weight': 0, 'width': 1}), (0, 5, {'weight': 0, 'width': 1}), (0, 6, {'weight': 0, 'width': 1}), (0, 7, {'weight': 1, 'width': 1}), (0, 9, {'weight': 0, 'width': 1}), (1, 2, {'weight': 18, 'width': 1}), (1, 3, {'weight': 0, 'width': 1}), (1, 4, {'weight': 0, 'width': 1}), (1, 6, {'weight': 0, 'width': 1}), (1, 8, {'weight': 0, 'width': 1}), (1, 9, {'weight': 0, 'width': 1}), (1, 10, {'weight': 5, 'width': 1}), (1, 11, {'weight': 0, 'width': 1}), (2, 4, {'weight': 0, 'width': 1}), (2, 3, {'weight': 0, 'width': 1}), (2, 6, {'weight': 8, 'width': 1}), (2, 8, {'weight': 0, 'width': 1}), (2, 9, {'weight': 2, 'width': 1}), (2, 11, {'weight': 0, 'width': 1}), (2, 10, {'weight': 0, 'width': 1}), (3, 4, {'weight': 18, 'width': 1}), (3, 5, {'weight': 0, 'width': 1}), (3, 6, {'weight': 0, 'width': 1}), (3, 7, {'weight': 0, 'width': 1}

UnboundLocalError: local variable 'coauthorID' referenced before assignment

Display Network

In [ ]:
nt = Network()
# populates the nodes and edges data structures
nt.from_nx(network)
nt
nt.show('testModel.html')